In [73]:
import numpy as np
import pandas as pd
import itertools
import folium
import time
import openrouteservice as ors
import sys
import urllib.parse
import pulp
import datetime

In [74]:
tfr_time = pd.read_csv("time1.csv")
tfr_list = pd.read_csv("test1.csv")
place_list = tfr_list.to_numpy().tolist()
print(place_list)

customer_count = len(place_list)
loc_loc_time = tfr_time.to_numpy().tolist()
num_places_time = len(tfr_time)

print(tfr_time)
cost = [[86401 for i in range(customer_count)] for j in range(customer_count)]
for i in range(num_places_time):
    cost[int(loc_loc_time[i][0])][int(loc_loc_time[i][1])] = float(loc_loc_time[i][2])
    cost[int(loc_loc_time[i][1])][int(loc_loc_time[i][0])] = float(loc_loc_time[i][2])
cost = np.array(cost)
cost

[['長津田駅', 35.5319589, 139.49173], ['河口湖ビジネス&リゾート+サワホテル', 35.4946247, 138.7589732], ['Bakery&Table+箱根（ベーカリー＆テーブル）', 35.2001708, 139.0321096], ['うなぎ+桜家', 35.1191933, 138.9138269], ['箱根神社', 35.2040652, 139.0266838], ['大涌谷', 35.2414681, 139.0227329], ['早雲山駅', 35.2459732, 139.0357894], ['箱根ガラスの森美術館', 35.2609093, 139.0183711], ['三嶋大社', 35.1195374, 138.9178982], ['三島スカイウォーク', 35.1547315, 138.9820768]]
    出発地点  行先  移動時間[s]
0      0   1   4969.9
1      0   2   4580.9
2      0   3   4332.1
3      0   4   4617.9
4      0   6   4270.1
..   ...  ..      ...
59     9   3    654.5
60     9   4   1204.5
61     9   6   1643.3
62     9   7   1925.7
63     9   8    641.1

[64 rows x 3 columns]


array([[8.6401e+04, 4.9699e+03, 4.5809e+03, 4.3321e+03, 4.6179e+03,
        8.6401e+04, 4.2701e+03, 4.0214e+03, 4.3665e+03, 4.7723e+03],
       [4.9699e+03, 8.6401e+04, 3.4610e+03, 2.8882e+03, 3.5799e+03,
        8.6401e+04, 2.8957e+03, 2.6471e+03, 2.9226e+03, 3.2084e+03],
       [4.5809e+03, 3.4610e+03, 8.6401e+04, 1.4439e+03, 3.1190e+02,
        8.6401e+04, 6.2470e+02, 9.0710e+02, 1.4305e+03, 1.1055e+03],
       [4.3321e+03, 2.8882e+03, 1.4439e+03, 8.6401e+04, 1.6517e+03,
        8.6401e+04, 1.9816e+03, 2.0352e+03, 3.4400e+01, 6.5450e+02],
       [4.6179e+03, 3.5799e+03, 3.1190e+02, 1.6517e+03, 8.6401e+04,
        8.6401e+04, 6.6170e+02, 9.4410e+02, 1.5295e+03, 1.2045e+03],
       [8.6401e+04, 8.6401e+04, 8.6401e+04, 8.6401e+04, 8.6401e+04,
        8.6401e+04, 8.6401e+04, 8.6401e+04, 8.6401e+04, 8.6401e+04],
       [4.2701e+03, 2.8957e+03, 6.2470e+02, 1.9816e+03, 6.6170e+02,
        8.6401e+04, 8.6401e+04, 3.4190e+02, 1.9683e+03, 1.6433e+03],
       [4.0214e+03, 2.6471e+03, 9.0710e+0

In [75]:
start_place_time = (datetime.time(5,00),datetime.time(23,00))
lunch_time = datetime.time(12,00)
lim_time = (lunch_time.hour *60 + lunch_time.minute) *60 - (start_place_time[0].hour *60 + start_place_time[0].minute) *60
lim_time3 = (start_place_time[1].hour *60 + start_place_time[1].minute) *60 - (start_place_time[0].hour *60 + start_place_time[0].minute) *60
count_place = customer_count
lunch_count = 2

count_remove_place1 = list(range(0,count_place))
print(count_remove_place1)
del count_remove_place1[2:lunch_count+2]
count_remove_place2 = count_remove_place1.copy()
del count_remove_place1[0]
del count_remove_place2[1]

print(count_remove_place1,count_remove_place2)

# 問題の宣言
problem = pulp.LpProblem("CVRP", pulp.LpMaximize)

# 決定変数
x = [[[pulp.LpVariable("x%s_%s,%s"%(i,j,k), cat="Binary") \
       if i != j else None for k in range(2)]for j in range(count_place)] for i in range(count_place)]

# 目的関数
problem += pulp.lpSum(x[i][j][k] if i != j else 0
                        for k in range(2) for j in range(count_place) for i in range (count_place))

# 制約
# (2)式，各顧客の場所に訪れるのは1台の車両で1度である
for j in range(count_place):
    problem += pulp.lpSum(x[i][j][k] if i != j else 0 for i in range(count_place) for k in range(2)) <= 1 
    problem += pulp.lpSum(x[j][i][k] if i != j else 0 for i in range(count_place) for k in range(2)) <= 1 

#(3)式, depotから出発して，depotに戻ってくる

# 出発地から必ず出る
problem += pulp.lpSum(x[0][j][0] for j in range(count_place)) == 1
problem += pulp.lpSum(x[i][0][0] for i in range(count_place)) == 0
problem += pulp.lpSum(x[i][1][0] for i in range(count_place)) == 0

problem += pulp.lpSum(x[1][j][1] for j in range(count_place)) == 0
problem += pulp.lpSum(x[i][1][1] for i in range(count_place)) == 1

# 指定した昼ご飯の場所に到着する
problem += pulp.lpSum(x[i][j][0] for i in range(count_place) for j in range(2,lunch_count + 2)) == 1
problem += pulp.lpSum(x[i][j][0] for j in range(count_place) for i in range(2,lunch_count + 2)) == 0

problem += pulp.lpSum(x[i][j][1] for i in range(2,count_place) for j in range(2,lunch_count + 2)) == 0
problem += pulp.lpSum(x[i][j][1] for j in range(2,count_place) for i in range(2,lunch_count + 2)) == 1

# 午前の最後と午後の最初は一緒
for l in range(2,lunch_count + 2):
    problem += pulp.lpSum(x[l][j][1] for j in range(2,count_place)) - pulp.lpSum(x[i][l][0] for i in range(count_place)) == 0

#(4)式, ある顧客の所に来る車両数と出る車両数が同じ
for j in count_remove_place1:
    problem += pulp.lpSum(x[i][j][0] if i != j else 0 for i in range(count_place)) -  pulp.lpSum(x[j][i][0] for i in range(count_place)) == 0
for j in count_remove_place2:
    problem += pulp.lpSum(x[i][j][1] if i != j else 0 for i in range(count_place)) -  pulp.lpSum(x[j][i][1] for i in range(count_place)) == 0

#(5)式, 各車両において最大容量を超えない
problem += pulp.lpSum((3600 + cost[i][j]) * x[i][j][0] if i != j else 0 for i in range(count_place) for j in range (1,count_place)) <= lim_time +7200
problem += pulp.lpSum((3600 + cost[i][j]) * x[i][j][0] if i != j else 0 for i in range(count_place) for j in range (1,count_place)) >= lim_time +3600

problem += pulp.lpSum((3600 + cost[i][j]) * x[i][j][1] if i != j else 0 for i in range(count_place) for j in range (1,count_place)) + \
    pulp.lpSum((3600 + cost[i][j]) * x[i][j][0] if i != j else 0 for i in range(count_place) for j in range (1,count_place)) <= lim_time3 +3600


#(6)式, 部分巡回路除去制約
subtours = []
for i in range(2,count_place):
        subtours += itertools.combinations(range(1,count_place), i)

for s in subtours:
    problem += pulp.lpSum(x[i][j][k] if i !=j else 0 for i, j in itertools.permutations(s,2) for k in range(2)) <= len(s) - 1

#最適化問題を解く
#最適解が出たら終了
if problem.solve() == 1:
    print('目的関数値:', pulp.value(problem.objective))
else:
    print(problem)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 4, 5, 6, 7, 8, 9] [0, 4, 5, 6, 7, 8, 9]
目的関数値: 7.0


In [76]:
basyo_num_list = []
basyo_list = [0]
jikan = 0

for k in range(2):
    print(k)
    for i in range(count_place):
        for j in range(count_place):
            if i != j and pulp.value(x[i][j][k]) == 1:
                print(i,j)
                jikan += 3600 + cost[i][j]
                basyo_num_list.append([i,j])


print(basyo_num_list)
print("~",start_place_time[1].hour - start_place_time[0].hour,"\n最適化",(jikan-3600)/3600)

print(jikan-3600)
for i in range(len(basyo_num_list)):
    for j in range(len(basyo_num_list)):
        if basyo_num_list[j][0] == basyo_list[-1]:
            basyo_list.append(basyo_num_list[j][1])
basyo_list

0
0 9
4 3
7 8
8 4
9 7
1
3 6
6 1
[[0, 9], [4, 3], [7, 8], [8, 4], [9, 7], [3, 6], [6, 1]]
~ 18 
最適化 10.674694444444444
38428.9


[0, 9, 7, 8, 4, 3, 6, 1]